In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import qutip as qt
from tqdm import tqdm
from matplotlib import cm
import scipy.integrate as integrate
from scipy.special import hermite 
from math import factorial

: 

In [ ]:
def cf_point(rho,beta):
    N = int(rho.dims[0][0])
    Dbeta = qt.displace(N,beta)
    return qt.expect(rho,Dbeta)

def cf_array(rho,betas):
    N = int(rho.dims[0][0])
    array_shape = betas.shape
    betas = np.ravel(betas)
    result = np.zeros(len(betas),dtype=complex)
    for ii in range(len(betas)):
        result[ii] = cf_point(rho,betas[ii])
    return result.reshape(array_shape)

def CatMaker(alpha,n,N):
    psi = qt.coherent(N,alpha)
    for i in range(n-1):
        psi += qt.coherent(N,np.exp(1j *2*np.pi/n*(i+1)) * alpha)
    return psi.unit()

do_plot = 1
check_norm = 1

def GKPmaker(Delta,xmax,pmax,N):

    a = qt.destroy(N)
    ad = a.dag()
    x = (a+ad)/np.sqrt(2)
    p = -1j*(a-ad)/np.sqrt(2)
   
    # Delta = .2    #.34  #.6
    Kxg = (np.sqrt(np.pi) * p).cosm() * (np.sqrt(np.pi) * Delta**2 * x).cosm()+ np.sin(np.pi *Delta**2/2) * (np.sqrt(np.pi) * p).cosm()
    # Kxe = -np.cos(np.pi * Delta**2 /2) * (np.sqrt(np.pi) * p).sinm() + 1j * (np.sqrt(np.pi) * p).cosm()*(np.sqrt(np.pi) * Delta**2 * x).sinm()
    Kzg = (np.sqrt(np.pi) * x).cosm() * (np.sqrt(np.pi) * Delta**2 * -p).cosm()+ np.sin(np.pi *Delta**2/2) * (np.sqrt(np.pi) * x).cosm()
    # Kze = -np.cos(np.pi * Delta**2 /2) * (np.sqrt(np.pi) * x).sinm() + 1j * (np.sqrt(np.pi) * x).cosm()*(np.sqrt(np.pi) * Delta**2 * -p).sinm()
   
    Kgg = Kzg*Kxg
    # Kge = Kzg*Kxe
    # Keg = Kze*Kxg
    # Kee = Kze*Kxe

    Es, vs = (-Kgg.dag()*Kgg).eigenstates()
    # p0 = vs[0] * vs[0].dag() + vs[1] * vs[1].dag()
    # p1 = vs[2] * vs[2].dag() + vs[3] * vs[3].dag()
    # p2 = vs[4] * vs[4].dag() + vs[5] * vs[5].dag()

    rho= vs[0].proj()

    # if do_plot:
    #     xvec = np.linspace(0,xmax,100)
    #     pvec = np.linspace(0,pmax,100)
    #     X,Y = np.meshgrid(xvec,pvec)
    #     Z = X+1.0j*Y
    #     N = rho.dims[0][0]
    #     cf =cf_array(rho,Z)
    #     fig, (ax1, ax2) = plt.subplots(1, 2,figsize = (8,3))
    #     ax1.contourf(xvec, pvec, np.abs(cf)**2, 100,  cmap=cm.seismic , norm = nrm)
    #     ax1.set_title(r'$|C(\alpha)|^2$')
    #     ax2.contourf(xvec, pvec, np.abs(cf)**4, 100,  cmap=cm.seismic , norm = nrm)
    #     ax2.set_title(r'$|C(\alpha)|^4$')

    # if check_norm:
    #     def cf2(x,y):
    #         beta = x + 1j * y
    #         return np.abs(cf_point(rho,beta))**2
    #     i2 = 4*integrate.nquad(cf2, [[0, xmax],[0, pmax]])[0]/np.pi

    return rho

def M_boson(rho,xmax,pmax):
    i2 = 0

    if do_plot:
        xvec = np.linspace(0,xmax,100)
        pvec = np.linspace(0,pmax,100)
        X,Y = np.meshgrid(xvec,pvec)
        Z = X+1.0j*Y
        N = rho.dims[0][0]
        cf =cf_array(rho,Z)
       
        plt.figure(1)
        fig, (ax1, ax2) = plt.subplots(1, 2,figsize = (8,3))
        ax1.contourf(xvec, pvec, np.abs(cf)**2, 100,  cmap=cm.seismic , norm = nrm)
        ax1.set_title(r'$|C(\alpha)|^2$')
        ax2.contourf(xvec, pvec, np.abs(cf)**4, 100,  cmap=cm.seismic , norm = nrm)
        ax2.set_title(r'$|C(\alpha)|^4$')

    if check_norm:
        def cf2(x,y):
            beta = x + 1j * y
            return np.abs(cf_point(rho,beta))**2
        i2 = 4*integrate.nquad(cf2, [[0, xmax],[0, pmax]])[0]/np.pi



    def cf4(x,y):
        beta = x + 1j * y
        return np.abs(cf_point(rho,beta))**4
    i4 = 4*integrate.nquad(cf4, [[0, xmax],[0, pmax]])[0]/np.pi
   
    M = -np.log2(i4/np.pi)
    return M, i2,i4


def M_boson_no_symm(rho,xmax,pmax):
    i2 = 0

    if do_plot:
        xvec = np.linspace(-xmax,xmax,100)
        pvec = np.linspace(-pmax,pmax,100)
        X,Y = np.meshgrid(xvec,pvec)
        Z = X+1.0j*Y
        N = rho.dims[0][0]
        cf =cf_array(rho,Z)
       
        plt.figure(1)
        fig, (ax1, ax2) = plt.subplots(1, 2,figsize = (8,3))
        ax1.contourf(xvec, pvec, np.abs(cf)**2, 100,  cmap=cm.seismic , norm = nrm)
        ax1.set_title(r'$|C(\alpha)|^2$')
        ax2.contourf(xvec, pvec, np.abs(cf)**4, 100,  cmap=cm.seismic , norm = nrm)
        ax2.set_title(r'$|C(\alpha)|^4$')

    if check_norm:
        def cf2(x,y):
            beta = x + 1j * y
            return np.abs(cf_point(rho,beta))**2
        i2 = integrate.nquad(cf2, [[-xmax, xmax],[-pmax, pmax]])[0]/np.pi



    def cf4(x,y):
        beta = x + 1j * y
        return np.abs(cf_point(rho,beta))**4
    i4 = integrate.nquad(cf4, [[-xmax, xmax],[-pmax, pmax]])[0]/np.pi
   
    M = -np.log2(i4/np.pi)
    return M, i2,i4

def rand_H1(n_max,N):
    H= 0 * qt.qeye(N)
    a = qt.destroy(N)
    ad = a.dag()
    for n in np.linspace(1,n_max,n_max):
        n = int(n)
        eps =(np.random.rand() +1j*np.random.rand())/np.sqrt(2)
        H+=eps * a**n + np.conjugate(eps)*ad**n
    # thresh = 1e-4
    # super_threshold_indices = H.full() > thresh
    # H.full()[super_threshold_indices] = 0
    return H

def rand_H2(n_max,N):
    H= 0 * qt.qeye(N)
    a = qt.destroy(N)
    ad = a.dag()

    for n in np.linspace(1,n_max,n_max):
        n =  int(n)
        # print('order(a)=',n)
        eps =(np.random.rand() +1j*np.random.rand())/np.sqrt(2)
        H += eps * a**n + np.conjugate(eps)*ad**n
        for m in np.linspace(1,n,n):
            m =  int(m)
            if n+m<=n_max:
                if n==m:
                    H += np.random.rand() * (ad**n * a**m)
                else:
                    H += np.random.rand() * (ad**n * a**m) + np.random.rand() * (ad**m * a**n)
                # print('order(ada)=', m+n)
    # thresh = 1e-4
    # super_threshold_indices = H.full() > thresh
    # H.full()[super_threshold_indices] = 0
    return H
    
def rand_H_fast(n_max,N):
    a = qt.destroy(N)
    ad = a.dag()
    H= 0 * qt.qeye(N)
    eps =(np.random.rand() +1j*np.random.rand())/np.sqrt(2)
    for n in np.linspace(1,n_max,n_max):
        n = int(n)
        H+=np.random.rand() * (ad**n * a**n) + eps * a**n + np.conjugate(eps)*ad**n
    return H

def rand_U(n_max,dt,N):
   return (-1j*rand_H_fast(n_max,N)*dt).expm()

def M_arrow(n_max,Steps,xmax,dt,N):
    psi_t = []
    psi_0 = qt.fock(N,0)
    psi_t.append(psi_0)
    for i in range(Steps):
        psi_t.append(rand_U(n_max,dt,N) * psi_t[-1])
    M = []
    pmax = xmax
    for i in tqdm(range(Steps)):
        rho = psi_t[i].proj()
        Mb = M_boson_no_symm(rho,xmax,pmax)[0]
        print(Mb)
        M.append(Mb)
    return M





def rand_H_symplectic(n_max,N):
    a = qt.destroy(N)
    ad = a.dag()
    eps1 = np.random.rand()*np.exp(2*np.pi*1j*np.random.rand())
    eps2 = np.random.rand()*np.exp(2*np.pi*1j*np.random.rand())

    H= eps1 * a + np.conjugate(eps1)*ad
    H+= np.random.rand() * (ad * a) 
    H+= eps2 * a**2 + np.conjugate(eps2)*ad**2
    return H

def rand_U_symplectic(n_max,dt,N):
   return (-1j*rand_H_symplectic(n_max,N)*dt).expm()

def M_arrow_symplectic(n_max,Steps,xmax,dt,N):
    psi_t = []
    psi_0 = qt.fock(N,0)
    psi_t.append(psi_0)
    for i in range(Steps):
        psi_t.append(rand_U_symplectic(n_max,dt,N) * psi_t[-1])
    M = []
    pmax = xmax
    for i in tqdm(range(Steps)):
        rho = psi_t[i].proj()
        Mb = M_boson_no_symm(rho,xmax,pmax)[0]
        print(Mb)
        M.append(Mb)
    return M




### TOOLS FOR GKP IN FOCK BASIS

J_max = 50 #12
N_max = 150 #60

def pos_coeffs(n, x):
    c = np.sqrt(2**n * factorial(n) * np.sqrt(np.pi))
    return np.exp(-x**2/2) * hermite(n)(x) / c

def gkp_coeffs(n, s, Delta):
    offs = 0 if s==0 else 1/2
    a = 0
    for j in range(-J_max, J_max+1, 1):
        a += pos_coeffs(n, 2*np.sqrt(np.pi)*(j+offs))
    a *= np.exp(-Delta**2 * n)
    return a

def gkp_state(Delta, s, N):
    psi = np.zeros(N)
    for n in range(N):
        if n < N_max:
            psi[n] = gkp_coeffs(n, s, Delta)
        else:
            n = 0
    norm = np.sqrt(np.sum(psi**2))
    return (psi / norm).reshape([1,N])


# def GKP_W_Maker(delta):
#     states = {}
#     states['+Z'] = gkp_state(delta, 0, Ns)
#     #     states['-Z'] = gkp_state(Delta, 1, Ns)
#     states = {s : qt.Qobj(v, type='ket') for (s,v) in states.items()}
#     rho_psi= states['+Z'].proj()
    
#     meanN = (rho_psi * qt.create(Ns)*qt.destroy(Ns)).tr()
#     return qt.wigner(rho_psi, xvec, pvec), meanN
